In [1]:
from torchsummary import summary
from thop import profile

from model import CM_Encoder

{'T1 HB': [[0.2962, 0.2962, 0.2962], [0.2527, 0.2527, 0.2527]], 'T2': [[0.2055, 0.2055, 0.2055], [0.1727, 0.1727, 0.1727]], 'out': [[0.2, 0.2, 0.2], [0.1644, 0.1644, 0.1644]]}
Loaded pretrained weights for efficientnet-b2


In [2]:
from torch import nn
import torch
from model import ShuffleV1Block
d_model = 1408
from efficientnet_pytorch import EfficientNet


class cm(nn.Module):
    def __init__(self):
        super().__init__()
        group = 8
        self.conv3d = torch.nn.Conv3d(1408, d_model, (3, 1, 1), groups=group)
        # self.conn11 = torch.nn.Conv2d(3, 1, (1, 1))
        # self.conn12 = torch.nn.Conv2d(1408, 1408, (1, 1))
        self.shuffle_block = ShuffleV1Block(1408, 1408, group=group,
                                            mid_channels=1408, ksize=1, stride=1)
        self.shuffle_block2 = ShuffleV1Block(1408, 1408, group=group,
                                            mid_channels=1408, ksize=1, stride=1)
        self.shuffle_block3 = ShuffleV1Block(1408, 1408, group=group,
                                            mid_channels=1408, ksize=1, stride=1)
        # self.eca = eca_layer(None)

        self.block = nn.Sequential(
            # nn.BatchNorm2d(d_model),
            nn.AdaptiveAvgPool2d(output_size=1),
        )
        self.proj = nn.Sequential(
            # nn.Dropout(.5),
            # nn.Linear(d_model, d_model),
            # nn.SiLU(),
            nn.Dropout(.3),
            nn.Linear(d_model, 2),
        )
    def forward(self, r):
        fmap = self.conv3d(r).squeeze(2)

        #Shuffle block 
        # fmap = fmap.permute(0, 2, 1, 3, 4).contiguous()
        # fmap = fmap.view(bsz, -1, fmap.size(-2), fmap.size(-1))
        fmap = self.shuffle_block(fmap)
        fmap = self.shuffle_block2(fmap)
        fmap = self.shuffle_block3(fmap)
        return fmap

In [3]:
effnet = EfficientNet.from_pretrained('efficientnet-b2',).cuda()
9,109,994
summary(effnet, (3, 200, 200))
inputs = torch.randn(1, 3, 200, 200)
effnet = EfficientNet.from_pretrained('efficientnet-b2',)
flops, params = profile(effnet, inputs=(inputs, ))
flops, params

Loaded pretrained weights for efficientnet-b2
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         ZeroPad2d-1          [-1, 3, 202, 202]               0
Conv2dStaticSamePadding-2         [-1, 32, 100, 100]             864
       BatchNorm2d-3         [-1, 32, 100, 100]              64
MemoryEfficientSwish-4         [-1, 32, 100, 100]               0
         ZeroPad2d-5         [-1, 32, 102, 102]               0
Conv2dStaticSamePadding-6         [-1, 32, 100, 100]             288
       BatchNorm2d-7         [-1, 32, 100, 100]              64
MemoryEfficientSwish-8         [-1, 32, 100, 100]               0
          Identity-9             [-1, 32, 1, 1]               0
Conv2dStaticSamePadding-10              [-1, 8, 1, 1]             264
MemoryEfficientSwish-11              [-1, 8, 1, 1]               0
         Identity-12              [-1, 8, 1, 1]               0
Conv2dStaticSamePadding-13        

(17764640.0, 1476568.0)

In [4]:
model = cm().cuda()
summary(model, (1408, 3, 7, 7))
inputs = torch.randn(1, 3, 200, 200)
model = cm()
flops, params = profile(model, inputs=(torch.rand(1, 1408, 3, 7, 7), ))
flops, params

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [-1, 1408, 1, 7, 7]         744,832
            Conv2d-2           [-1, 1408, 7, 7]         247,808
       BatchNorm2d-3           [-1, 1408, 7, 7]           2,816
              SiLU-4           [-1, 1408, 7, 7]               0
    ShuffleV1Block-5           [-1, 1408, 7, 7]               0
            Conv2d-6           [-1, 1408, 7, 7]         247,808
       BatchNorm2d-7           [-1, 1408, 7, 7]           2,816
              SiLU-8           [-1, 1408, 7, 7]               0
    ShuffleV1Block-9           [-1, 1408, 7, 7]               0
           Conv2d-10           [-1, 1408, 7, 7]         247,808
      BatchNorm2d-11           [-1, 1408, 7, 7]           2,816
             SiLU-12           [-1, 1408, 7, 7]               0
   ShuffleV1Block-13           [-1, 1408, 7, 7]               0
Total params: 1,496,704
Trainable param

(73338496.0, 1496704.0)

In [5]:
# from thop import profile
# flops, params = profile(model, inputs=(torch.randn(1408, 3, 7, 7) ))
291491200.0
73338496.0

73338496.0

In [6]:
class cm(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv3d = torch.nn.Conv3d(1408, d_model, (3, 1, 1))
        self.block = nn.Sequential(
            # nn.BatchNorm2d(d_model),
            nn.AdaptiveAvgPool2d(output_size=1),
        )
        self.proj = nn.Sequential(
            # nn.Dropout(.5),
            # nn.Linear(d_model, d_model),
            # nn.SiLU(),
            nn.Dropout(.3),
            nn.Linear(d_model, 2),
        )
    def forward(self, r):
        fmap = self.conv3d(r).squeeze(2)
        return fmap
model = cm().cuda()
summary(model, (1408, 3, 7, 7))
5307392
inputs = torch.randn(1, 3, 200, 200)
model = cm()
flops, params = profile(model, inputs=(torch.rand(1, 1408, 3, 7, 7), ))
flops, params


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [-1, 1408, 1, 7, 7]       5,948,800
Total params: 5,948,800
Trainable params: 5,948,800
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.79
Forward/backward pass size (MB): 0.53
Params size (MB): 22.69
Estimated Total Size (MB): 24.01
----------------------------------------------------------------
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv3d'>.
[INFO] Register count_adap_avgpool() for <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'>.
[WARN] Cannot find rule for <class 'torch.nn.modules.container.Sequential'>. Treat it as zero Macs and zero Params.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[WARN] Cannot find rule for <class '__main__.c

(291491200.0, 5948800.0)